In [1]:
import h5py 
import numpy as np
import os
import time
from numpy import linalg as LA
from random import randint

In [2]:
h = [0.2]
v = [1,1,0]
pos = [2,3,4]
h_new = 10
obstacles = np.array([[1,2,3,1],[1,2,3,1],[1,2,3,1]])

FOVx = 120
FOVy=10
Resolutionx =10
Resolutiony = 10
numberObj = 10
Radius = 1


In [3]:
import glob, os, os.path
#from path import path
#path =os.path.realpath('Savefiles')
import glob, os
os.chdir("/home/atilla/Documents/DeepLearning/OpticFlow/Trash")
files = [file for file in glob.glob("*.hdf5")]
print(files)

test = 'mytestf.hdf5'
matching = [True for s in files if test == s]
print(matching)
    
#     print ('true')
#     print(file)
#p = path("/Documents/DeepLearning/Automate/")
# #print(os.path.dirname(os.path.realpath(__file__)))

# for file in glob.glob("*.hdf5"):
#     print(file)
[entry for entry in os.scandir('.') if entry.is_file()]


[]
[]


[<DirEntry 'Simulation Optic Flow.ipynb'>,
 <DirEntry 'sphereIntersection.ipynb'>,
 <DirEntry 'savefiles.ipynb'>,
 <DirEntry 'version4.py'>,
 <DirEntry 'Version3.ipynb'>,
 <DirEntry 'SimulationOpticFlow.py'>]

In [4]:
files = [file for file in glob.glob("*.hdf5")]
test= 'mytestfile.hdf5'
#test = 'test'+time.strftime('%F-%T')+'.hdf5'
exist = False

for s in files:
    if test == s:
        exist = True
        print('exists')
        #f.close()
        f = h5py.File(test)
        if 'obstacles' in f:
            r= obstacles.shape
            print(f['obstacles'].shape)
            f['obstacles'].resize(r)
            f['obstacles'][...] = obstacles
            data= f['obstacles']
            print(f['obstacles'][:])
            #groupName = ''

        else:    
            data = f.create_dataset('obstacles', data =obstacles, dtype = 'f', chunks=True )


if exist == False:
    f =h5py.File(test, 'w')
    data = f.create_dataset('obstacles', data=obstacles,dtype='f', chunks = True)

    
data.attrs['Resolution']= [Resolutionx, Resolutiony]
data.attrs['NumberObjects']=numberObj
data.attrs['FOV']=[FOVx,FOVy]    
f.close()
%ls

mytestfile16.hdf5            SimulationOpticFlow.py    version4.py
savefiles.ipynb              sphereIntersection.ipynb
Simulation Optic Flow.ipynb  Version3.ipynb


In [53]:
maxVel = 20
maxHeading = np.pi*2
SF2 = 1 # safety factor
area = 10
samples =3

velocity = [[randint(-maxVel, maxVel),randint(-maxVel,maxVel),0] for i in range(samples)]
velocity = [np.divide(vel, LA.norm(vel)) for vel in velocity]
heading = [np.random.sample(1)*maxHeading for i in range(samples)]
position =[np.array([randint(0,area),randint(0,area),0]) for i in range(samples)]


v,p,h = [[[0,0,0], 0 , 0] for i in range(3)]
h

[[0, 0, 0], 0, 0]

In [6]:
layout = np.array([[pos, vel, head] 
                   for pos in position
                   for vel in velocity
                    for head in heading])
layout.shape


(27, 3)

In [11]:
f =h5py.File(test)
data = f['obstacles']
num =1
for i in layout:
    name = 'tryout_'+str(num)
    #print(i[0],i[1],i[2], name)
    ofx = np.zeros((20,2))
    ofy = np.ones((20,2))
    new_heading = 5
    if name in f:
        grp = f[name]
    else:
        grp = f.create_group(name)
    if "ofx" in grp:
        r = ofx.shape
        grp['ofx'].resize(r)
        grp['ofx'][...]=ofx
    else:
        grp.create_dataset("ofx",data= ofx,dtype='f', chunks = True)
    if "ofy" in grp:
        r = ofy.shape
        grp['ofy'].resize(r)
        grp['ofy'][...]=ofy
    else:
        grp.create_dataset("ofy",data= ofy, dtype='f', chunks = True)
    if 'new_heading' in grp:
        grp['new_heading'][...]=new_heading
    else:
        grp.create_dataset("new_heading", data =new_heading )
    grp.attrs['position']=i[0]
    grp.attrs['velocity']= i[1]
    grp.attrs['heading']=i[2]
    #print(grp['ofx'])
    num = num +1
    #grp = f.create_group(name)
    #print(num,'num')


In [14]:
listf=[]
def printname(name):
    listf.append(name)

f.visit(printname)


In [9]:
f['tryout2_1'].attrs['velocity']


array([-0.99227788, -0.12403473,  0.        ])

In [48]:
listf = [key for key in f.keys() if 'tryout' in key]
xOF = [f[x]['ofx'][:] for x in listf]

np.array(xOF).shape


(27, 20, 2)